In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-30T16:31:08.683191-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: NVIDIA H100 80GB HBM3
  GPU 1: NVIDIA H100 80GB HBM3
  GPU 2: NVIDIA H100 80GB HBM3
  GPU 3: NVIDIA H100 80GB HBM3
  GPU 4: NVIDIA H100 80GB HBM3
  GPU 5: NVIDIA H100 80GB HBM3
  GPU 6: NVIDIA H100 80GB HBM3
  GPU 7: NVIDIA H100 80GB HBM3



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%watermark --iversions

xgboost: 1.7.5
optuna : 3.1.1
logging: 0.5.1.2
pandas : 1.5.3
numpy  : 1.23.5
dask   : 2023.3.2



In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-30 16:31:16,404 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:31:16,404 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:31:16,404 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:31:16,404 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:31:16,418 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:31:16,418 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:31:16,419 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:31:16,419 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 16:31:16,419 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 16:31:16,419 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

In [7]:
X_d = delayed(pd.read_csv)(f'../input/X.csv')
X = dd.from_delayed(X_d)
    
X_test_d = delayed(pd.read_csv)(f'../input/X_test.csv')
X_test = dd.from_delayed(X_test_d)
    
    
train_y = X['loss']
X = X[X.columns.difference(['loss'])]

In [9]:
RANDOM_STATE = 2016
params = {
        'min_child_weight': 1,
        'eta': 0.01,
        'colsample_bytree': 0.5,
        'max_depth': 12,
        'subsample': 0.8,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'verbose_eval': True,
        'seed': RANDOM_STATE
    }

xgtrain = xgb.dask.DMatrix(client, X, train_y, enable_categorical=True)
xgtest = xgb.dask.DMatrix(client, X_test, enable_categorical=True)

/home/dgxuser/.local/lib/python3.10/site-packages/xgboost/core.py:617: FutureWarning: Pass `weight` as keyword args.
  warnings.warn(msg, FutureWarning)


TypeError: Not supported type for data.<class 'distributed.client.Client'>